In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import nibabel as nib
from tensorflow import keras
import tensorflow_addons as tfa

In [2]:
def data_gen():
    df = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/train.csv")
    for i in range(df.shape[0]):
        path = "../input/cervical-ct-preprocessing/" + str(df.iloc[i]["StudyInstanceUID"]) +".nii.gz"
        try:
            ct = nib.load(path)
            ct = ct.get_fdata()
            yield ({"input_1": ct}, {'co': df.iloc[i]["patient_overall"], 
                                    'c1': df.iloc[i]["C1"],
                                    'c2': df.iloc[i]["C2"],
                                    'c3': df.iloc[i]["C3"],
                                    'c4': df.iloc[i]["C4"],
                                    'c5': df.iloc[i]["C5"],
                                    'c6': df.iloc[i]["C6"],
                                    'c7': df.iloc[i]["C7"],})
        except:
            continue

In [12]:
dataset = tf.data.Dataset.from_generator(
     data_gen,
     ({"input_1":tf.float32}, 
     {'co':tf.int8,
     'c1':tf.int8,
     'c2':tf.int8,
     'c3':tf.int8,
     'c4':tf.int8,
     'c5':tf.int8,
     'c6':tf.int8,
     'c7':tf.int8}),
    ({"input_1":tf.TensorShape([100,100,60])}, 
     {"co": tf.TensorShape([]), 
     "c1": tf.TensorShape([]),
     "c2": tf.TensorShape([]),
     "c3": tf.TensorShape([]),
     "c4": tf.TensorShape([]),
     "c5": tf.TensorShape([]),
     "c6": tf.TensorShape([]),
     "c7": tf.TensorShape([])})
)

In [13]:
dataset = dataset.batch(12)
dataset = dataset.shuffle(buffer_size = 20, seed=3)
dataset=dataset.prefetch(1)


In [18]:
inputA = keras.layers.Input(shape=(100,100,60), name="input_1")
X = keras.layers.Conv2D(2048, 3)(inputA)
X = keras.layers.MaxPooling2D()(X)
X = keras.layers.Conv2D(1024, 3)(X)
X = keras.layers.MaxPooling2D()(X)
X = keras.layers.Conv2D(512, 3)(X)
X = keras.layers.Conv2D(512, 3)(X)
X = keras.layers.MaxPooling2D()(X)
X = keras.layers.Conv2D(128, 3)(X)
X = keras.layers.MaxPooling2D()(X)

X = keras.layers.Flatten()(X)
X = keras.layers.BatchNormalization()(X)
X = keras.layers.Dense(1024 ,activation = 'relu')(X)
X = keras.layers.Dense(512 ,activation = 'relu')(X)
X = keras.layers.Dense(256 ,activation = 'relu')(X)
X = keras.layers.BatchNormalization()(X)
X = keras.layers.Dense(128 ,activation = 'relu')(X)
X = keras.layers.Dense(16 ,activation = 'relu')(X)
outputo = keras.layers.Dense(1 ,activation = 'sigmoid', name="co")(X)
output1 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c1")(X)
output2 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c2")(X)
output3 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c3")(X)
output4 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c4")(X)
output5 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c5")(X)
output6 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c6")(X)
output7 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c7")(X)
                                               
model = keras.models.Model(inputs=inputA, outputs=[outputo,output1,output2,output3,output4,output5,output6,output7])

In [19]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100, 100, 60 0                                            
__________________________________________________________________________________________________
conv2d_10 (Conv2D)              (None, 98, 98, 2048) 1107968     input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_10 (MaxPooling2D) (None, 49, 49, 2048) 0           conv2d_10[0][0]                  
__________________________________________________________________________________________________
conv2d_11 (Conv2D)              (None, 47, 47, 1024) 18875392    max_pooling2d_10[0][0]           
____________________________________________________________________________________________

In [20]:
opt = tfa.optimizers.AdamW(
    weight_decay=1e-4,
    learning_rate = 0.001,
    beta_1 = 0.9,
    beta_2 = 0.999,
    epsilon = 1e-07,
    name = 'AdamW',
)
loss = tf.keras.losses.BinaryCrossentropy(
    reduction=tf.keras.losses.Reduction.AUTO,
    name='binary_crossentropy'
)
metrics = [tf.keras.metrics.AUC()]
model.compile(optimizer=opt,
              loss={"co": loss, 
                   "c1": loss,
                   "c2": loss,
                   "c3": loss,
                   "c4": loss,
                   "c5": loss,
                   "c6": loss,
                   "c7": loss,},
                metrics={"co": metrics, 
                   "c1": metrics,
                   "c2": metrics,
                   "c3": metrics,
                   "c4": metrics,
                   "c5": metrics,
                   "c6": metrics,
                   "c7": metrics,})


In [8]:
!mkdir best_model

In [21]:
model_checkpoint_callback_LASSO = tf.keras.callbacks.ModelCheckpoint(
    filepath = './best_model/model',
    monitor="val_loss",
    save_best_only=True,
    save_weights_only=True,
    mode="auto",
)

In [22]:
model.fit(dataset.take(105), validation_data = dataset.skip(105), epochs=30, callbacks=[model_checkpoint_callback_LASSO])

Epoch 1/30


Exception ignored in: <generator object data_gen at 0x7f31d82b0250>
RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object data_gen at 0x7f3151cf2ed0>
RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object data_gen at 0x7f314ef81750>
RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object data_gen at 0x7f314ef818d0>
RuntimeError: generator ignored GeneratorExit


    105/Unknown - 57s 318ms/step - loss: 3.5962 - co_loss: 0.7130 - c1_loss: 0.3785 - c2_loss: 0.5564 - c3_loss: 0.2740 - c4_loss: 0.2753 - c5_loss: 0.3636 - c6_loss: 0.4611 - c7_loss: 0.5742 - c7_c6_c5_c4_c3_c2_c1_co_auc_1: 0.6798

Exception ignored in: <generator object data_gen at 0x7f31f31d2f50>
RuntimeError: generator ignored GeneratorExit


105/105 [==============================] - 93s 662ms/step - loss: 3.5962 - co_loss: 0.7130 - c1_loss: 0.3785 - c2_loss: 0.5564 - c3_loss: 0.2740 - c4_loss: 0.2753 - c5_loss: 0.3636 - c6_loss: 0.4611 - c7_loss: 0.5742 - c7_c6_c5_c4_c3_c2_c1_co_auc_1: 0.6798 - val_loss: 3.4453 - val_co_loss: 0.7409 - val_c1_loss: 0.3581 - val_c2_loss: 0.5243 - val_c3_loss: 0.1788 - val_c4_loss: 0.3280 - val_c5_loss: 0.3108 - val_c6_loss: 0.4402 - val_c7_loss: 0.5642 - val_c7_c6_c5_c4_c3_c2_c1_co_auc_1: 0.7044
Epoch 2/30
105/105 [==============================] - 74s 658ms/step - loss: 2.9440 - co_loss: 0.6924 - c1_loss: 0.2585 - c2_loss: 0.4464 - c3_loss: 0.1472 - c4_loss: 0.2025 - c5_loss: 0.2888 - c6_loss: 0.4065 - c7_loss: 0.5018 - c7_c6_c5_c4_c3_c2_c1_co_auc_1: 0.7496 - val_loss: 3.1039 - val_co_loss: 0.6895 - val_c1_loss: 0.2757 - val_c2_loss: 0.5271 - val_c3_loss: 0.1969 - val_c4_loss: 0.2099 - val_c5_loss: 0.2766 - val_c6_loss: 0.4278 - val_c7_loss: 0.5004 - val_c7_c6_c5_c4_c3_c2_c1_co_auc_1: 0.74

...

Exception ignored in: <generator object data_gen at 0x7f2cf55c54d0>
RuntimeError: generator ignored GeneratorExit


Epoch 29/30
105/105 [==============================] - 72s 643ms/step - loss: 1.6985 - co_loss: 0.3212 - c1_loss: 0.1767 - c2_loss: 0.2081 - c3_loss: 0.0953 - c4_loss: 0.1352 - c5_loss: 0.1712 - c6_loss: 0.2422 - c7_loss: 0.3485 - c7_c6_c5_c4_c3_c2_c1_co_auc_1: 0.9366 - val_loss: 3.0975 - val_co_loss: 0.6906 - val_c1_loss: 0.3362 - val_c2_loss: 0.4841 - val_c3_loss: 0.1689 - val_c4_loss: 0.1906 - val_c5_loss: 0.2819 - val_c6_loss: 0.4177 - val_c7_loss: 0.5274 - val_c7_c6_c5_c4_c3_c2_c1_co_auc_1: 0.8169
Epoch 30/30


Exception ignored in: <generator object data_gen at 0x7f2cf55c5550>
RuntimeError: generator ignored GeneratorExit


105/105 [==============================] - 72s 643ms/step - loss: 1.6338 - co_loss: 0.3104 - c1_loss: 0.1782 - c2_loss: 0.1971 - c3_loss: 0.0949 - c4_loss: 0.1407 - c5_loss: 0.1664 - c6_loss: 0.2224 - c7_loss: 0.3237 - c7_c6_c5_c4_c3_c2_c1_co_auc_1: 0.9397 - val_loss: 3.4963 - val_co_loss: 0.8653 - val_c1_loss: 0.2717 - val_c2_loss: 0.5576 - val_c3_loss: 0.1680 - val_c4_loss: 0.2183 - val_c5_loss: 0.3356 - val_c6_loss: 0.4864 - val_c7_loss: 0.5933 - val_c7_c6_c5_c4_c3_c2_c1_co_auc_1: 0.7854


Exception ignored in: <generator object data_gen at 0x7f3174c3c8d0>
RuntimeError: generator ignored GeneratorExit


In [23]:
model.load_weights("./best_model/model").expect_partial()

In [24]:
!mkdir final_model

In [25]:
model.save('./final_model/model')

2022-10-06 19:28:48.749063: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
